In [24]:
import pandas as pd
import numpy as np
from sklearn.naive_bayes import GaussianNB

In [32]:
data = pd.read_csv('DT19_Datos_Var_Rpta_train.csv')

In [33]:
data.head()

,id,f_analisis,var_rpta,segmento
0,1,201803,0,4
1,2,201604,0,0
2,3,201608,0,5
3,4,201706,0,4
4,5,201703,0,4


In [34]:
colnames = data.columns.values.tolist()

In [35]:
colnames

['id', 'f_analisis', 'var_rpta', 'segmento']

In [47]:
predictors = [colnames[3]]

In [48]:
target = colnames[2]

In [49]:
data["is_train"] = np.random.uniform(0,1, len(data))<= 0.8

In [50]:
train, test = data[data["is_train"]==True],data[data["is_train"]==False]

In [51]:
gnb = GaussianNB()

In [52]:
modelnb = gnb.fit(train[predictors],train[target])

In [53]:
print(modelnb.score(train[predictors],train[target]))

0.9282385072039512


In [54]:
predsnb = modelnb.predict(test[predictors])
pd.crosstab(test[target],predsnb,rownames=["Actual"],colnames=["Predictions"])

Predictions,0
Actual,
0,14252
1,1137
